In [5]:
# We need a graph representation with k commodities, sources and sinks.

# Sets

There are three kinds of nodes: forests $N^F$, sawmills $N^S$, and transshipment hubs $N^H$. Forests and sawmills act as sources and sinks (respectively) in the real-world representation of the problem. The nodes for the hubs represent potential locations to build hubs. Therefore, our graph consists of:

\begin{align*}
G =&\; (N, A) \\
N =&\; N^F \cup N^S \cup N^H 
\end{align*}

There are two types of vehicles available:
\begin{align*}
V =&\; \{\text{Truck, Transporter}\}
\end{align*}

Next we define the **arcs**: according to the exercise, transporting logs between sawmills and between forests is prohibited. Transporting "backwards" (from sawmills to forests, sawmills to hubs, and hubs to forests) is not possible. We further prohibit transporting between hubs.
\begin{align*}
    A = (N^F \times N^H) \cup (N^H \times N^S) \cup (N^F \times N^S)
\end{align*}

Up until now, the model is only specified for a single type of logs. We have $K$ different types of logs, where $K\in \mathbb{N}$.

In further notation, every $k$ will denote one type of log, such that $k \in \{1, \dots, K\}$ and $k\in \mathbb{N}$.

We need to adapt the graph accounting for the multiple types of logs. In principle, we assume that each forest can produce every type of log and each sawmill can process every type of log. Also, we introduce artificial sinks and sources for each type of log to facilitate modelling:

\begin{align*}
S = \{S^1, \dots, S^k, \dots, S^K\} &\quad  \text{ (set of artificial sources)}\\
T = \{T^1, \dots, T^k, \dots, T^K\} &\quad \text{ (set of artificial sinks)}
\end{align*}

The artificial sinks and sources are connected to each forest and sawmill respectively. That means that there exist arcs 

\begin{align*}
S^k \times N^F \text{ and } N^S\times T^k \quad | \quad \forall k
\end{align*}

However, these connections are not part of $A$ and will be treated seperately.

# Variables

There are two decision variables. The first is the number of units of a certain type of logs $k$ transported through arc $(i,j)$:

$$x_{ij}^k \in \mathbb{N}_0$$

The second variable is the decision on whether to build a hub at a potential location.

\begin{align*}
y_i =
  \begin{cases}
    1 & \text{if hub } i \text{ is built}\\
    0 & \text{otherwise}.
  \end{cases} \quad  \forall i \in N^H
\end{align*}

# Parameters

## Node-based parameters

Some parameters are related to the nodes. Hubs have fixed operation costs and unit costs of handling logs.

\begin{align*}
o_i \in \mathbb{R}^{+} \quad | \quad \forall i \in N^H & \text{ (operation cost of hub $i$)}\\
h_{i}^k \in \mathbb{R}^{+} \quad | \quad \forall i \in N^H & \text{ (handling cost per unit of type $k$ at hub $i$)}\\
\end{align*}

Both forests and sawmills have a maximum capacity for each type of log, which are supply $s_i^k$ (for forests) and demand $d_i^k$ (for sawmills) respectively.

\begin{align*}
s_i^k \in \mathbb{Z}^+ \quad | \quad \forall i \in N^F \\
d_i^k \in \mathbb{Z}^+ \quad | \quad \forall i \in N^S
\end{align*}

## Arc-based parameters

Every arc can be travelled by at most one of a specified type of vehicle. That is, all arcs that connect to forests can only be travelled by transporters; arcs between hubs and sawmills can only be travelled by transporters. Which vehicle is used on an arc $(i,j)$ is denoted by $v_{ij}$, so the capacity of an arc is:

$$\kappa_{ij} = \kappa_{v_{ij}} \quad | \quad \forall (i,j) \in A,\; v \in V$$

where $\kappa_{Truck} > \kappa_{Transporter}$.

The capacity of an arc that connects the source of each type of log to a forest is equal to the supply of that forest:

$$\kappa_{S^ki} = s_i \quad | \quad \forall i \in N^F, \forall k$$ 

In similar fashion, the capacity of the arcs connecting sawmills with the sink for each type of log is equal to the demand of this sawmill:

$$\kappa_{iT^k} = d_i \quad | \quad \forall i \in N^S, \forall k$$

The total cost of transporting a unit of a log of type $k$ on an arc $(i,j)$ is equal to the cost of the vehicle travelling that arc to travel that arc:

\begin{align*}
c_{v_{ij}}^k = c_{ij}^k \in \mathbb{Z}^+
\end{align*}

Since every arc can only be travelled by one type of vehicle (and one vehicle at a time), we do not need to make the cost vehicle-specific. Instead, transport costs have to be calculated before initialising the model, by taking into account vehicle characteristics like fuel use and arc characteristics like length, inclination, and possibly other factors.

The total cost of transporting a type of log on an arc will be the cost per unit of that type, multiplied by the number of units of logs transported (will be defined further down in the objective).

The transport costs from sources and to sinks are $0$. This is because the sources and sink nodes are artificial.

$$c_{S^kj}^k =  c_{iT^k}^k = 0, \; \forall i \in N^S, \; \forall j \in N^F,\;\forall k \in K $$


## Other parameters

The price $r^k$ of a log of type $k$ occurs for each unit that it is delivered to its corresponding sink $T_k$.

$$r^k \in \mathbb{R}^+$$

 The condition on when the price is counted towards the objective function, combined with the zero costs of the arcs between $N^S$ and $S^k$ and the flow conservation constraints will guarantee that each unit of a log will be modelled to reach its sink.

# Objective function

The first part of the objective function is the **revenue generated** by all the commodities.

$$
\text{revenue} = \sum_{k=1}^K \sum_{i \in N^s} x_{iT^k}^k r^k 
$$

Note that the revenue is only generated for logs reaching the sink of the respective log type. However, the constraints will make sure that every log that leaves a source will also reach the sink.

The rest of the objective function accounts for different costs. First off, there are handling costs that occur at each hub for all units of this type of log reaching the hub from all forest nodes. To get the full handling costs, we need to sum that for all types of logs and all hubs.

$$\text{handling costs} = \sum_{k = 1}^{K} \Big[ \sum_{j \in N^H} \big[ \sum_{i \in N^F} x_{ij}^k h_j^k \big] \Big]$$

We also have to substract the costs that stem from operating hubs (the activation or operating costs). 

$$
\text{operation costs} = \sum_{i \in N^H}o_i y_i
$$

Besides , transporting the logs also produces costs. The transport costs occur at every arc $(i,j) \in A$ and are dependent on the number of logs for each type $k$ of logs transported.

$$\text{transport costs} = \sum_{\forall (i,j) \in A} x_{ij}^kc_{ij}^k $$

Note that, since the arcs from $S$ and to $T$ are not in $A$, logs flowing from sources and sinks will not produce any costs.

The ultimate objective function is 

$$max\text{(revenue - handling costs - operation costs - transport costs)}$$ 

and can be written as:

\begin{align*}
 \\
 \max \bigg( & \sum_{k=1}^K \sum_{i \in N^s} x_{iT^k}^k r^k && \text{ (revenue) } \\
- &  \sum_{k = 1}^{K} \Big[ \sum_{j \in N^H} \big[ \sum_{i \in N^F} x_{ij}^k h_j^k \big] \Big] && \text{ (handling costs at hubs) } \\
- &  \sum_{i \in N^H}o_i y_i && \text{ (fixed operation costs for hubs)} \\
- &  \sum_{\forall (i,j) \in A} x_{ij}^kc_{ij}^k \bigg) && \text{ (transport costs)} 
\end{align*}

# Constraints

## Make sure that only the right kind of commodity can reach the sink / source

That is, for any arc connecting a sawmill $N^S$ and a sink $T^k$, the number of logs of type $l \neq k$ flowing through that arc has to be 0.

\begin{align*}
x_{iT^k}^l = 0 \quad | \quad \forall i \in N^S, \; k \leq K, \; l \leq K, \; l \neq k
\end{align*}


The same holds for arcs between the sources $S^k$ and the forests $N^F$.

\begin{align*}
x_{S^ki}^l = 0 \quad | \quad \forall i \in N^F, \; k \leq K, \; l \leq K, \; l \neq k
\end{align*}

## The total number of logs of all types transported on an arc (i,j) cannot exceed the arc capacity $\kappa_{ij}$

$$\sum_{k=1}^K x_{ij}^k \leq \kappa_{ij} \quad | \quad \forall (i,j) \in A$$

Note that, since $A$ does not include the arcs from sources and to sinks, we do not impose any capacity restrains on these arcs. The only restraints there will be the supply of forests and demands of sawmills; see the third constraint.

## Flow conservation constraints

The capacity of arcs connecting the forests with the sources and the sawmills with the sinks has already been established as the supply / demand of the respective forest / sawmill. In the flow conservation, we have to account for the possibility of the demand not matching the supply; therefore, we have to allow for the net inflow of any forest being smaller or equal than the supply of this forest (and similarly, for sawmills). Mathematically, this possibility can be expressed as:

$$
\sum_{\forall (i,j) \in A}x_{ij}^k \leq s_i^k \quad \forall k \in K, \; i \in N^F\\
$$

This means that the total amount of units of a commodity $k$ leaving any forest is smaller or equal than the supply of the forest. The same holds for sawmills, where

$$
\sum_{\forall (i,j) \in A}x_{ji}^k \leq d_i^k \quad \forall k \in K, \; i \in N^S\\
$$

so that the total amount of units of a commodity $k$ reaching any sawmill is smaller or equal than the demand of that sawmill to commodity $k$.

However, we instead decide to impose zero flow conservation on all nodes $i \in N$. That means that the net inflow to every node $i \in N$ has to be zero. This is written as 

$$
\sum_{\forall (i,j) \in A}x_{ij}^k = \sum_{\forall (j,i) \in A}x_{ji}^k \quad \forall i \in N,\; k \in K
$$

To account for possibilities where supply and demand do not match, we allow for "residual" supply or demand in sources and sinks. That is, the amount of units of a commodity $k$ flowing from the source $S^k$ to a forest $i \in N^F$ is smaller or equal than the supply of this commodity of all forests, $s_i^k$. The same holds for sinks and sawmills. 

$$
x_{S^ki}^k \leq s_i^k \quad \forall k \in K, \; i \in N^F \\
x_{iT^k}^k \leq d_i^k \quad \forall k \in K, \; i \in N^S \\
$$ 

# Extensions

Hubs can only take logs from at most $p_i$ forest areas, $i \in N^F$

\begin{align*}
u_ij =
  \begin{cases}
    1 & \text{if hub } j \text{ take logs from forest} i\\
    0 & \text{otherwise}.
  \end{cases} \quad  \forall i \in N^F, j \in N^H
\end{align*}

$$
\sum_{(i,j) \in A} u_{ij} \leq p_i \quad \forall  \quad i \in N^F, j\in N^H.
$$